<a href="https://colab.research.google.com/github/ramajoballester/mil_deeplearning/blob/main/1_Introduccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 1. Introducción al Deep Learning

# Introducción

## Instalación de librerías

In [2]:
! pip install -q wandb

     |████████████████████████████████| 1.8 MB 3.3 MB/s 
     |████████████████████████████████| 181 kB 54.8 MB/s 
     |████████████████████████████████| 145 kB 62.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [ ]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git clone https://github.com/d2l-ai/d2l-en.git

In [ ]:
!git clone https://github.com/ramajoballester/mil_deeplearning.git

Cloning into 'mil_deeplearning'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 35 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.


# **Text**

#### Text

In [1]:
!pip install --upgrade -q gspread

In [2]:
import gspread
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.auth import default

In [3]:
creds, _ = default()
gc = gspread.authorize(creds)

In [34]:
url = 'https://docs.google.com/spreadsheets/d/1rDJQgihE4bvVNGmPjT8sH5mCRrbOfNcEFf8KZoxZ870/edit?usp=sharing'
worksheet = gc.open_by_url(url)

In [35]:
a = float(worksheet.sheet1.get('C5')[0][0])

In [36]:
a

0.954782

In [26]:
row = 5
col = 3
value = 0.954782
worksheet.sheet1.update_cell(row, col, value);